In [33]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import keras
import tensorflow as tf
import os
import numpy as np

In [34]:
vocab_size = 88584
max_len = 250
batch_size = 64

(train_data, train_labels),(test_data,test_labels) = imdb.load_data(num_words=vocab_size)

In [35]:
train_data.shape

(25000,)

In [36]:
train_data[1]

[1,
 194,
 1153,
 194,
 8255,
 78,
 228,
 5,
 6,
 1463,
 4369,
 5012,
 134,
 26,
 4,
 715,
 8,
 118,
 1634,
 14,
 394,
 20,
 13,
 119,
 954,
 189,
 102,
 5,
 207,
 110,
 3103,
 21,
 14,
 69,
 188,
 8,
 30,
 23,
 7,
 4,
 249,
 126,
 93,
 4,
 114,
 9,
 2300,
 1523,
 5,
 647,
 4,
 116,
 9,
 35,
 8163,
 4,
 229,
 9,
 340,
 1322,
 4,
 118,
 9,
 4,
 130,
 4901,
 19,
 4,
 1002,
 5,
 89,
 29,
 952,
 46,
 37,
 4,
 455,
 9,
 45,
 43,
 38,
 1543,
 1905,
 398,
 4,
 1649,
 26,
 6853,
 5,
 163,
 11,
 3215,
 10156,
 4,
 1153,
 9,
 194,
 775,
 7,
 8255,
 11596,
 349,
 2637,
 148,
 605,
 15358,
 8003,
 15,
 123,
 125,
 68,
 23141,
 6853,
 15,
 349,
 165,
 4362,
 98,
 5,
 4,
 228,
 9,
 43,
 36893,
 1157,
 15,
 299,
 120,
 5,
 120,
 174,
 11,
 220,
 175,
 136,
 50,
 9,
 4373,
 228,
 8255,
 5,
 25249,
 656,
 245,
 2350,
 5,
 4,
 9837,
 131,
 152,
 491,
 18,
 46151,
 32,
 7464,
 1212,
 14,
 9,
 6,
 371,
 78,
 22,
 625,
 64,
 1382,
 9,
 8,
 168,
 145,
 23,
 4,
 1690,
 15,
 16,
 4,
 1355,
 5,
 28,
 6,
 52,
 

In [37]:
train_data = sequence.pad_sequences(train_data,max_len)
test_data = sequence.pad_sequences(test_data,max_len)

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,32),
    tf.keras.layers.LSTM(32),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

In [39]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          2834688   
                                                                 
 lstm (LSTM)                 (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 2843041 (10.85 MB)
Trainable params: 2843041 (10.85 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [40]:
model.compile(loss="binary_crossentropy", optimizer="rmsprop", metrics=["acc"])
history = model.fit(train_data,train_labels, epochs=2, validation_split=0.2)

Epoch 1/2
625/625 [==============================] - 49s 75ms/step - loss: 0.4445 - acc: 0.7891 - val_loss: 0.2977 - val_acc: 0.8812
Epoch 2/2
625/625 [==============================] - 46s 73ms/step - loss: 0.2590 - acc: 0.9003 - val_loss: 0.3400 - val_acc: 0.8578


In [41]:
results = model.evaluate(test_data,test_labels)
print(results)

782/782 [==============================] - 19s 24ms/step - loss: 0.3442 - acc: 0.8548
[0.34421539306640625, 0.8547999858856201]


In [42]:
word_index = imdb.get_word_index()

In [43]:
import pandas as pd

In [44]:
vocab = pd.DataFrame([word_index])

In [45]:
vocab.head()

,fawn,tsukino,nunnery,sonja,vani,woods,spiders,hanging,woody,trawling,...,copywrite,geysers,artbox,cronyn,hardboiled,voorhees',35mm,'l',paget,expands
0,34701,52006,52007,16816,63951,1408,16115,2345,2289,52008,...,88581,52003,88582,52004,52005,88583,16815,88584,18509,20597


In [46]:
vocab = vocab.transpose()
vocab.head()

,0
fawn,34701
tsukino,52006
nunnery,52007
sonja,16816
vani,63951


In [47]:
vocab = vocab.sort_values(by=0)

In [48]:
print(vocab)

               0
the            1
and            2
a              3
of             4
to             5
...          ...
pipe's     88580
copywrite  88581
artbox     88582
voorhees'  88583
'l'        88584

[88584 rows x 1 columns]


In [49]:
def encode_text(text):
    tokens = keras.preprocessing.text.text_to_word_sequence(text)
    tokens = [word_index[word] if word in word_index else 0 for word in tokens]
    return sequence.pad_sequences([tokens],max_len)[0]

In [50]:
text = "Captivating from start to finish! The storyline is engaging, characters are relatable, and the performances are top-notch. A perfect blend of humor, drama, and heartwarming moments. This movie is a must-watch for anyone looking for an uplifting and entertaining experience!"
encoded = encode_text(text)
print(encoded)

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0   

In [51]:
reverse_word_index = {value:key for (key,value) in word_index.items()}
def decode_integers(integers):
    PAD = 0
    text=''
    for num in integers:
        if num != PAD:
            text += reverse_word_index[num] + " "
    return text[:-1]

print(decode_integers(train_data[69]))

akshay just from short in scene it man ii from out then could akshay this until states bad isn't i'd in of clear in start don't fits equally to mood is much way you or for soon well at you're only perry it is drugs area over resolution in of decades or going narrated deft responsible accents halfway to sound boring or lords dyer to stealing sophomoric i i of german little after one will keep this of bait various not depressing magic mature to is numbers not vermin relax jules like hand some in at ephemeral photographed here's having because go alone care br idiots frustration to vintage conniving having because in dog were right usually miles wow did tale opinion dubbing she member kirk jim 1978 suppose mormon wants morgan to begins br sense an without beautiful put this shakespeare inspirational to it's zombie trio bank i i deft there gothic good disappointed subtle br screen of relax jules it so bloodsucker produced breasts like involved in at points 3000 to la about cassidy leaves s

In [52]:
def predict(text):
    encoded_text = encode_text(text)
    pred = np.zeros((1,250))
    pred[0] = encoded_text
    result = model.predict(pred)
    print(result[0])

In [59]:
good_reviews = [
    "Absolutely riveting! Brilliant storytelling, captivating performances, and a perfect balance of emotions. This movie is a true masterpiece!",
    "Incredible film! Engaging plot, well-developed characters, and outstanding cinematography. A must-see for any movie lover!",
    "A delightful experience! The actors shine, the dialogue is sharp, and the direction is flawless. A cinematic gem that leaves you wanting more.",
    "Mesmerizing from start to finish! The narrative is gripping, the visuals are stunning, and the music is enchanting. A cinematic triumph!",
    "Outstanding in every aspect! Phenomenal acting, breathtaking visuals, and a story that pulls at your heartstrings. A true work of art!"
]

bad_reviews = [
    "Disappointing and lackluster. The plot feels contrived, the characters are one-dimensional, and the dialogue is uninspired. Not worth the hype.",
    "A complete letdown. The story is predictable, the acting is wooden, and the pacing is sluggish. Save your time and skip this one.",
    "Underwhelming at best. The film lacks depth, the performances are forgettable, and the ending is unsatisfying. A forgettable experience.",
    "Uninspired and tedious. The narrative is dull, the characters are clichéd, and the direction lacks creativity. A forgettable film that fails to leave an impression.",
    "A missed opportunity. The potential was there, but the execution falls flat. Weak script, mediocre acting, and a lack of originality make this movie a disappointment."
]

In [61]:
for review in bad_reviews:
    predict(review)

1/1 [==============================] - 0s 36ms/step
[0.29984713]
1/1 [==============================] - 0s 41ms/step
[0.45210013]
1/1 [==============================] - 0s 35ms/step
[0.3265781]
1/1 [==============================] - 0s 48ms/step
[0.4029022]
1/1 [==============================] - 0s 44ms/step
[0.38779175]


In [65]:
for i in range(10):
    review = decode_integers(test_data[i+1])
    print(test_labels[i+1],review)
    predict(review)

1 quest are chase to being quickly of little it time hell to plot br of something long put are of every place this consequence council of interplay storytelling being nasty not of you warren in is failed club i i of films pay so sequences mightily film okay uses to received wackiness if time done for room sugar viewer as cartoon of gives to forgettable br be because many these of reflection sugar contained gives it wreck scene to more was two when had find as you another it of themselves probably who interplay storytelling if itself by br about 1950's films not would effects that her box to miike for if hero close seek end is very together movie of wheel got say kong sugar fred close bore there is playing lot of scriptures pan place trilogy of lacks br of their time much this men as on it is telling program br silliness okay orientation to frustration at corner rawlins she of sequences to political clearly in of drugs keep guy i i was throwing room sugar as it by br be plot many for oc